# Homework 3 - Interactive Viz

In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import folium
import re
import sys

## European unemployment rates

_Go to the [eurostat](http://ec.europa.eu/eurostat/data/database) website and try to find a dataset that includes the european unemployment rates at a recent date._

   _Use this data to build a _[Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) _which shows the unemployment rate in Europe at a country level. Think about [the colors you use](https://carto.com/academy/courses/intermediate-design/choose-colors-1/),how you decided to [split the intervals into data classes](http://gisgeography.com/choropleth-maps-data-classification/) or which interactions you could add in order to make the visualization intuitive and expressive. Compare Switzerland's unemployment rate to that of the rest of Europe._

On the eurostat website, we found our dataset under 'Database by themes/Population and social conditions/Labour Market(labour)/Employment and unemployment/LFS main indicators/Unemployment - LFS adjusted series/Unemployment by sex and age - annual average'. It provides the unemployment rates up to 2016 for all european countries. On the contrary of its name, both sex and all ages are mixed.

In [ ]:
CSV_PATH = 'data/question1/une_rt_a_1_Data.csv'

europe_df = pd.read_csv(CSV_PATH) 

europe_df = europe_df.loc[europe_df['TIME'] == 2016]
europe_df = europe_df.loc[europe_df['UNIT'] == 'Percentage of active population']
europe_df = europe_df.drop(['TIME','AGE', 'SEX','UNIT','Flag and Footnotes'], axis = 1)
europe_df = europe_df[6:]
europe_df = europe_df[:30]
# Clean Germany's name
europe_df['GEO'] = europe_df['GEO'].str.replace(r"\(.*\)","")
europe_df['Value'] = europe_df['Value'].astype(float)
europe_df = europe_df[~(europe_df['GEO'] == 'Cyprus')]
europe_df = europe_df[~(europe_df['GEO'] == 'Malta')]

In [ ]:
for line in europe_df['GEO']:
    if line == 'Germany ':
        line = 'Germany'
europe_df.head()

In [ ]:
state_topo_path = r'topojson/europe.topojson.json'
topo_json_data = json.load(open(state_topo_path))

In [ ]:
length = europe_df.shape[0]
for i in range(0,51):
    present = 0
    for index, row in europe_df.iterrows():
        if row['GEO'] == topo_json_data['objects']['europe']['geometries'][i]['properties']['NAME']:
            present = 1
            presence_row = row
    if present == 0:
        if topo_json_data['objects']['europe']['geometries'][i]['properties']['NAME'] == 'Switzerland':
            europe_df.loc[-1] = [topo_json_data['objects']['europe']['geometries'][i]['properties']['NAME'],3.5]
        else:
            europe_df.loc[-1] = [topo_json_data['objects']['europe']['geometries'][i]['properties']['NAME'],-1]  # adding a row
        europe_df.index = europe_df.index + 1  # shifting index
        europe_df = europe_df.sort_index()
        #europe_df = europe_df.append({'GEO'}: topo_json_data['objects']['europe']['geometries'][i]['properties']['NAME'], {'Value'}: 'NaN', ignore_index=True)

europe_df = europe_df[~(europe_df['GEO'] == 'Germany')]

In [ ]:
def unemployment_color(state):
    if state == 'Germany':
        value = 4.1
    else:
        value = float(europe_df.loc[europe_df['GEO'] == state, 'Value'].values)
        
    if value < 0:
        return '#000000'
    elif value < 4:
        return '#ffffcc'
    elif value < 6:
        return '#ffffb3'
    elif value < 8:
        return '#ffcc99'
    elif value < 10:
        return '#ff9966'
    elif value < 12:
        return '#ff6600'  
    else:
        return '#ff0000'

    
results_map = folium.Map([54,17], tiles='cartodbpositron', zoom_start=4)
folium.TopoJson(
    data = topo_json_data,
    object_path = 'objects.europe',
    style_function=lambda feature: {
        'fillColor': unemployment_color(feature['properties']['NAME']),
        'color' : 'black',
        'weight' : 2,
        'opacity': 0.1}).add_to(results_map)

results_map

In [ ]:
state_geo = r'topojson/europe.topojson.json'
topo_json_data = json.load(open(state_geo))

us_map = folium.Map(location=[54,17], zoom_start=4)
us_map.choropleth(geo_data=topo_json_data, data=europe_df,
             columns=['GEO', 'Value'],
             key_on='feature.properties.NAME',
             fill_color='BuPu', fill_opacity=0.9, line_opacity=0.8,
             legend_name='Percentage of Unemployment (%)',
             topojson='objects.europe')
us_map

In [ ]:
state_geo = r'topojson/europe.topojson.json'
topo_json_data = json.load(open(state_geo))
swiss_unemploy_2016 = 3.5
for i in range(0,51):
    print topo_json_data['objects']['europe']['geometries'][i]['properties']['NAME']

In [ ]:
state_geo = r'topojson/europe.topojson.json'
topo_json_data = json.load(open(state_geo))

In [ ]:
europe_df = europe_df[~(europe_df['GEO'] == 'Germany')]
europe_df

In [ ]:
state_geo = r'topojson/europe.topojson.json'
topo_json_data = json.load(open(state_geo))

us_map = folium.Map(location=[54,17], zoom_start=4)
folium.GeoJson(topo_json_data).add(us_map)
us_map

## Swiss Cantons unemployement

_Go to the [amstat](https://www.amstat.ch) website to find a dataset that includes the unemployment rates in Switzerland at a recent date._

   > *HINT* Go to the `details` tab to find the raw data you need. If you do not speak French, German or Italian, think of using free translation services to navigate your way through. 

  _ Use this data to build another Choropleth map, this time showing the unemployment rate at the level of swiss cantons. Again, try to make the map as expressive as possible, and comment on the trends you observe._

  _ The Swiss Confederation defines the rates you have just plotted as the number of people looking for a job divided by the size of the active population (scaled by 100). This is surely a valid choice, but as we discussed one could argue for a different categorization._

  _ Copy the map you have just created, but this time don't count in your statistics people who already have a job and are looking for a new one. How do your observations change ? You can repeat this with different choices of categories to see how selecting different metrics can lead to different interpretations of the same data._

## Foreign workers in Switzerland


_Use the [amstat](https://www.amstat.ch) website again to find a dataset that includes the unemployment rates in Switzerland at recent date, this time making a distinction between *Swiss* and *foreign* workers._

  _The Economic Secretary (SECO) releases [a monthly report](https://www.seco.admin.ch/seco/fr/home/Arbeit/Arbeitslosenversicherung/arbeitslosenzahlen.html) on the state of the employment market. In the latest report (September 2017), it is noted that there is a discrepancy between the unemployment rates for *foreign* (`5.1%`) and *Swiss* (`2.2%`) workers. _

  _ Show the difference in unemployment rates between the two categories in each canton on a Choropleth map (*hint* The easy way is to show two separate maps, but can you think of something better ?). Where are the differences most visible ? Why do you think that is ?_

  _ Now let's refine the analysis by adding the differences between age groups. As you may have guessed it is nearly impossible to plot so many variables on a map. Make a bar plot, which is a better suited visualization tool for this type of multivariate data._

## Röstigraben

_ *BONUS*: using the map you have just built, and the geographical information contained in it, could you give a *rough estimate* of the difference in unemployment rates between the areas divided by the [Röstigraben](https://en.wikipedia.org/wiki/R%C3%B6stigraben)?_